In [7]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "seattle_sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "Rd": "Road",
            "raod": "road",
            "Ln": "Lane",
            "Rd.": "Road",
            "Ave": "Avenue",
            "seattle": "Seattle",
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

pprint.pprint(dict(audit(OSMFILE)))

def update_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            name = re.sub(street_type_re, mapping[street_type], name)

    return name

update_street = audit(OSMFILE) 



{'104': set(['Northeast State Highway 104']),
 '12': set(['US Highway 12']),
 '125': set(['Better Way SE Ste 125']),
 '15th': set(['South 15th']),
 '16th': set(['North 16th']),
 '2': set(['Highway 2']),
 '20': set(['State Route 20']),
 '534': set(['State Route 534']),
 '6th': set(['South 6th']),
 '9': set(['State Highway 9', 'State Route 9']),
 '98296': set(['196th Street Southeast, Snohomish, WA 98296']),
 'A': set(['Avenue A']),
 'Alley': set(['Post Alley',
               'Scotland Alley',
               "Seth's Alley",
               "Skyler's Alley",
               "Steve's Alley",
               'Stone Haven Alley']),
 'Av.': set(['Pacific Av.']),
 'B': set(['60th Lane Southeast unit B']),
 'Bend': set(['Wild Berry Bend']),
 'Broadway': set(['Broadway']),
 'CT': set(['SE 10TH CT']),
 'Center': set(['Parkplace Center']),
 'Chase': set(['Steeple Chase']),
 'Circle': set(['Briarwood Circle',
                'Bridlewood Circle',
                'Foxglove Circle',
                'Kirk